#### Patrick Duffy

National Renewable Energy Lab
COE Review 2020 - Floating
Last updated: 2021/9/24

In [1]:
import pandas as pd
from ORBIT import ProjectManager, load_config
from examples.construction_finance_param import con_fin_params

### Load and check the project configuration

In [2]:
floating_config = load_config("COE_2020_floating_project.yaml") 

print(f"Num turbines: {floating_config['plant']['num_turbines']}")
print(f"Turbine: {floating_config['turbine']}")
print(f"\nSite: {floating_config['site']}")
print(f"\nDesign phases: {floating_config['design_phases']}")
print(f"\nInstall phases: {list(floating_config['install_phases'].keys())}")

Num turbines: 75
Turbine: COE_2020_8MW

Site: {'depth': 739, 'distance': 189, 'distance_to_landfall': 36}

Design phases: ['ArraySystemDesign', 'ExportSystemDesign', 'MooringSystemDesign', 'OffshoreSubstationDesign', 'SemiSubmersibleDesign']

Install phases: ['ArrayCableInstallation', 'ExportCableInstallation', 'MooredSubInstallation', 'MooringSystemInstallation', 'OffshoreSubstationInstallation']


### Run

In [3]:
project = ProjectManager(floating_config)
d = 0.3 # includes 30% downtime factor during construction to match ORCA
project.run(availability=1-d)

ORBIT library intialized at 'c:\users\pduffy\documents\repos\wisdem_orbit\orbit\library'


### Top Level Outputs

In [4]:
print(f"Installation CapEx:  {project.installation_capex_per_kw:.2f}")
print(f"System CapEx:        {project.system_capex_per_kw:.2f}")
print(f"BOS CapEx:           {project.bos_capex_per_kw:.2f}")
print(f"Turbine CapEx:       {project.turbine_capex_per_kw:.2f}")

Installation CapEx:  550.11
System CapEx:        2368.32
BOS CapEx:           2918.43
Turbine CapEx:       1301.00


In [5]:
dct = project.capex_breakdown_per_kw
dct = {k:[v] for k,v in dct.items()}
df = pd.DataFrame.from_dict(dct, orient="columns")

soft_dct = con_fin_params(bos=project.bos_capex, 
               turbine_capex=project.turbine_capex,
               orbit_install_capex=project.installation_capex,
               plant_cap=600000)

for k,v in soft_dct.items():
    if k != "soft_capex":
        df[k] = v
    else:
        df["Soft"] = v
df = df.T
df.to_csv("COE_floating_result.csv")

In [10]:
project.capex_breakdown_per_kw

{'Array System': 119.81010899777947,
 'Export System': 126.32423101563334,
 'Substructure': 1267.3571614999998,
 'Mooring System': 689.0318508205083,
 'Offshore Substation': 165.7985,
 'Array System Installation': 50.39122044319276,
 'Export System Installation': 190.76978591036053,
 'Substructure Installation': 145.310573630137,
 'Mooring System Installation': 150.9613910632747,
 'Offshore Substation Installation': 12.678772271145903,
 'Turbine': 1301.0,
 'Soft': 645.0,
 'Project': 252.08333333333334}

In [6]:
x = project.phases["SemiSubmersibleDesign"]
x.detailed_output

{'stiffened_column_mass': 1067.8916,
 'stiffened_column_cost': 3331821.792,
 'truss_mass': 569.2796,
 'truss_cost': 3557997.4999999995,
 'heave_plate_mass': 321.63919999999996,
 'heave_plate_cost': 2010244.9999999998,
 'secondary_steel_mass': 170.868,
 'secondary_steel_cost': 1238793.0}

In [8]:
print(project.installation_costs)

{'ArrayCableInstallation': 21218312.58614099, 'ExportCableInstallation': 113141808.65919928, 'MooredSubInstallation': 76075397.26027396, 'MooringSystemInstallation': 63613784.2465751, 'OffshoreSubstationInstallation': 5604084.353881278}


In [9]:
print(project.system_costs)

{'ArrayCableInstallation': 71886065.39866768, 'ExportCableInstallation': 75794538.60938, 'MooredSubInstallation': 760414296.9, 'MooringSystemInstallation': 413419110.492305, 'OffshoreSubstationInstallation': 99479100.0}


In [10]:
print(project.bos_capex)

1700646498.5064235


### Phase Specific Outputs

In [11]:
print(f"Substructure Installation:  {project.phases['MooredSubInstallation'].installation_capex/1e6:.0f} M")
print(f"Substructure Procurement:  {project.phases['MooredSubInstallation'].system_capex/1e6:.0f} M")

Substructure Installation:  76 M
Substructure Procurement:  760 M


In [12]:
print(f"Mooring System Installation:   {project.phases['MooringSystemInstallation'].installation_capex/1e6:.0f} M")
print(f"Mooring System Procurement:   {project.phases['MooringSystemInstallation'].system_capex/1e6:.0f} M")

Mooring System Installation:   64 M
Mooring System Procurement:   413 M


In [13]:
# print(f"Turbine Installation:  {project.phases['TurbineInstallation'].installation_capex/1e6:.0f} M")

In [14]:
print(f"Array System Installation: {project.phases['ArrayCableInstallation'].installation_capex/1e6:.0f} M")
print(f"Array System Procurement:  {project.phases['ArrayCableInstallation'].system_capex/1e6:.0f} M")

Array System Installation: 21 M
Array System Procurement:  72 M


### Installation Actions

In [15]:
print(project.phase_times)

{'ArrayCableInstallation': 2718.2244580323763, 'ExportCableInstallation': 397.96586151368757, 'MooredSubInstallation': 8609.4, 'MooringSystemInstallation': 9109.499999999909, 'OffshoreSubstationInstallation': 164.72979999999998}


In [16]:
pd.DataFrame(project.actions)

,cost_multiplier,agent,action,duration,cost,level,time,phase,location,phase_name,max_waveheight,max_windspeed,transit_speed,site_depth,num_vessels
0,0.5,Array Cable Installation Vessel,Mobilize,72.000,1.800000e+05,ACTION,0.000,ArrayCableInstallation,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.5,Export Cable Installation Vessel,Mobilize,72.000,1.800000e+05,ACTION,0.000,ExportCableInstallation,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,Onshore Construction,Onshore Construction,0.000,1.098817e+08,ACTION,0.000,ExportCableInstallation,Landfall,NaN,NaN,NaN,NaN,NaN,NaN
3,1.0,Mooring System Installation Vessel,Mobilize,168.000,7.000000e+05,ACTION,0.000,MooringSystemInstallation,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.5,Heavy Lift Vessel,Mobilize,72.000,7.500000e+05,ACTION,0.000,OffshoreSubstationInstallation,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3669,NaN,Mooring System Installation Vessel,Install Mooring Line,3.695,1.539583e+04,ACTION,9085.110,MooringSystemInstallation,NaN,MooringSystemInstallation,NaN,NaN,NaN,NaN,NaN
3670,NaN,Mooring System Installation Vessel,Position Onsite,2.000,8.333333e+03,ACTION,9087.110,MooringSystemInstallation,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3671,NaN,Mooring System Installation Vessel,Perform Mooring Site Survey,4.000,1.666667e+04,ACTION,9091.110,MooringSystemInstallation,NaN,MooringSystemInstallation,NaN,NaN,NaN,NaN,NaN
3672,NaN,Mooring System Installation Vessel,Install Suction Pile Anchor,14.695,6.122917e+04,ACTION,9105.805,MooringSystemInstallation,NaN,MooringSystemInstallation,NaN,NaN,NaN,NaN,NaN


In [17]:
d = pd.DataFrame(project.actions)
c = d.groupby('phase')
print(c['duration'].sum()/24)

phase
ArrayCableInstallation             116.259352
ExportCableInstallation             19.581911
MooredSubInstallation             1773.487500
MooringSystemInstallation          386.562500
OffshoreSubstationInstallation      19.454567
Name: duration, dtype: float64


In [18]:
da = pd.DataFrame(project.actions)
ca = da.groupby('action')
print(ca['duration'].sum()/24)

action
ActiveFeeder                                  1.965825
Attach Topside                                0.500000
Ballast to Operational Draft                 18.750000
Ballast to Towing Draft                      18.750000
Bolt TP                                       0.333333
Check Mooring Lines                          37.500000
Connect Mooring Lines                        68.750000
Crane Reequip                                 0.166667
Delay                                       214.937500
Delay: No Completed Assemblies Available     11.062500
Delay: No Substructures in Wet Storage      232.500000
Drive Monopile                                0.125000
Fasten Monopile                               1.000000
Fasten Topside                                1.000000
Install Mooring Line                         46.187500
Install Suction Pile Anchor                 183.687500
Lay/Bury Cable                               37.306118
Lift Topside                                  0.004167
Lif

In [19]:
print(project.capex_breakdown)

{'Array System': 71886065.39866768, 'Export System': 75794538.60938, 'Substructure': 760414296.9, 'Mooring System': 413419110.492305, 'Offshore Substation': 99479100.0, 'Array System Installation': 21218312.58614099, 'Export System Installation': 113141808.65919928, 'Substructure Installation': 76075397.26027396, 'Mooring System Installation': 63613784.2465751, 'Offshore Substation Installation': 5604084.353881278, 'Turbine': 736200000.0, 'Soft': 387000000, 'Project': 151250000.0}


In [20]:
print(project.capex_breakdown_per_kw)

{'Array System': 119.81010899777947, 'Export System': 126.32423101563334, 'Substructure': 1267.3571614999998, 'Mooring System': 689.0318508205083, 'Offshore Substation': 165.7985, 'Array System Installation': 35.363854310234984, 'Export System Installation': 188.56968109866548, 'Substructure Installation': 126.79232876712327, 'Mooring System Installation': 106.02297374429183, 'Offshore Substation Installation': 9.34014058980213, 'Turbine': 1227.0, 'Soft': 645.0, 'Project': 252.08333333333334}
